### Анализ убытков приложения ProcrastinatePRO+</div>


   <b>В данном исследовании проводем маркетинговый анализ причин убытка развлекательного приложения Procrastinate Pro+.</b>  
    
Имеем лог сервера с данными о посещениях приложения новыми пользователями, зарегистрировавшимися в период с 2019-05-01 по 2019-10-27, выгрузку их покупок за этот период, а также статистику рекламных расходов.  
    <b>
        Цель исследования:  </b>  
    - изучить, как люди пользуются продуктом,   
    - когда они начинают покупать,  
    - сколько денег приносит каждый клиент,   
    - откуда приходят пользователи и какими устройствами они пользуются,  
    - сколько стоит привлечение пользователей из различных рекламных каналов,  
    - когда расходы на привлечение клиента окупаются,   
    - какие факторы отричательно влияют на привлечение пользователей  
         <b>Ход исследования</b>  
    В нашем распоряжении три датасета:  
    Файл visits_info_short.csv хранит лог сервера с информацией о посещениях сайта,  
    orders_info_short.csv — информацию о заказах,  
    costs_info_short.csv — информацию о расходах на рекламу.   
    Проведем обзор данных и предобработку, после этого приступим к анализу



### Загрузим данные и подготовим их к анализу

Загрузим данные о визитах, заказах и рекламных расходах из CSV-файлов в переменные.

**Пути к файлам**

- визиты: `/datasets/visits_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/visits_info_short.csv);
- заказы: `/datasets/orders_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/orders_info_short.csv);
- расходы: `/datasets/costs_info_short.csv`. [Скачать датасет](https://code.s3.yandex.net/datasets/costs_info_short.csv).

Изучим данные и выполним предобработку. Есть ли в данных пропуски и дубликаты? Убедимся, что типы данных во всех колонках соответствуют сохранённым в них значениям. Обратим внимание на столбцы с датой и временем.

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta


try:
    visits = pd.read_csv('/datasets/visits_info_short.csv')
    orders = pd.read_csv('/datasets/orders_info_short.csv')
    costs = pd.read_csv('/datasets/costs_info_short.csv')
except:
    visits = pd.read_csv('visits_info_short.csv')
    orders = pd.read_csv('orders_info_short.csv')
    costs = pd.read_csv('costs_info_short.csv')

In [ ]:
visits.head()

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,United States,iPhone,organic,2019-05-01 02:36:01,2019-05-01 02:45:01
1,278965908054,United States,iPhone,organic,2019-05-01 04:46:31,2019-05-01 04:47:35
2,590706206550,United States,Mac,organic,2019-05-01 14:09:25,2019-05-01 15:32:08
3,326433527971,United States,Android,TipTop,2019-05-01 00:29:59,2019-05-01 00:54:25
4,349773784594,United States,Mac,organic,2019-05-01 03:33:35,2019-05-01 03:57:40


Таблица `visits` (лог сервера с информацией о посещениях сайта):

- **User Id** — уникальный идентификатор пользователя
- **Device** — категория устройства пользователя
- **Session start** — дата и время начала сессии
- **Session End** — дата и время окончания сессии
- **Channel** — идентификатор рекламного источника, из которого пришел пользователь
- **Region** - страна пользователя

In [ ]:
orders.head()

,User Id,Event Dt,Revenue
0,188246423999,2019-05-01 23:09:52,4.99
1,174361394180,2019-05-01 12:24:04,4.99
2,529610067795,2019-05-01 11:34:04,4.99
3,319939546352,2019-05-01 15:34:40,4.99
4,366000285810,2019-05-01 13:59:51,4.99


Таблица `orders` (хранит информацию о заказах)
- **User Id** — уникальный идентификатор пользователя,
- **Event Dt** — дата и время покупки,
- **Revenue** — сумма заказа.


In [ ]:
costs.head()

,dt,Channel,costs
0,2019-05-01,FaceBoom,113.3
1,2019-05-02,FaceBoom,78.1
2,2019-05-03,FaceBoom,85.8
3,2019-05-04,FaceBoom,136.4
4,2019-05-05,FaceBoom,122.1


Таблица costs(хранит информацию о расходах на рекламу.)
- **dt** — дата проведения рекламной кампании,
- **Channel** — идентификатор рекламного источника,
- **costs** — расходы на эту кампанию.


In [ ]:
# смотрим инфо
visits.info()
display('-------------')
orders.info()
display('-------------')
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User Id        309901 non-null  int64 
 1   Region         309901 non-null  object
 2   Device         309901 non-null  object
 3   Channel        309901 non-null  object
 4   Session Start  309901 non-null  object
 5   Session End    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


'-------------'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


'-------------'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   Channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


In [ ]:
# ищем пропуски в таблицах
display(visits.isna().sum())
display(orders.isna().sum())
display(costs.isna().sum())

User Id          0
Region           0
Device           0
Channel          0
Session Start    0
Session End      0
dtype: int64

User Id     0
Event Dt    0
Revenue     0
dtype: int64

dt         0
Channel    0
costs      0
dtype: int64

In [ ]:
# ищем дубликаты в таблице
display(visits.duplicated().sum())
display(orders.duplicated().sum())
costs.duplicated().sum()

0

0

0

In [ ]:
# заменим пробелы в названиях столбцов на нижнее подчеркивание и приведем к нижнему регистру
visits.columns = [x.lower().replace(' ', '_') for x in visits.columns]
orders.columns = [x.lower().replace(' ', '_') for x in orders.columns]
costs.columns = costs.columns.str.lower()

In [ ]:
# переводим тип столбцов из строк в дату
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
costs['dt'] = pd.to_datetime(costs['dt']).dt.date 

In [ ]:
# смотрим уникальные значения столбца user_id
visits['user_id'].value_counts()

33606575057     46
943775408561    36
901180916748    35
870784569391    34
764086596354    33
                ..
214203066007     1
369265191867     1
346271445800     1
133742530598     1
279181973476     1
Name: user_id, Length: 150008, dtype: int64

In [ ]:
# смотрим уникальные значения столбца region
visits['region'].value_counts()

United States    207327
UK                36419
France            35396
Germany           30759
Name: region, dtype: int64

In [ ]:
# смотрим уникальные значения столбца device
visits['device'].value_counts()

iPhone     112603
Android     72590
PC          62686
Mac         62022
Name: device, dtype: int64

In [ ]:
# смотрим уникальные значения столбца channel
visits['channel'].value_counts()

organic               107760
TipTop                 54794
FaceBoom               49022
WahooNetBanner         20465
LeapBob                17013
OppleCreativeMedia     16794
RocketSuperAds         12724
YRabbit                 9053
MediaTornado            8878
AdNonSense              6891
lambdaMediaAds          6507
Name: channel, dtype: int64

In [ ]:
# смотрим уникальные значения столбца user_id
orders['user_id'].value_counts

<bound method IndexOpsMixin.value_counts of 0        188246423999
1        174361394180
2        529610067795
3        319939546352
4        366000285810
             ...     
40207    651604369137
40208    275341387049
40209    374656616484
40210    168548862926
40211    329994900775
Name: user_id, Length: 40212, dtype: int64>

In [ ]:
# смотрим уникальные значения столбца channel
costs['channel'].value_counts()

FaceBoom              180
MediaTornado          180
RocketSuperAds        180
TipTop                180
YRabbit               180
AdNonSense            180
LeapBob               180
OppleCreativeMedia    180
WahooNetBanner        180
lambdaMediaAds        180
Name: channel, dtype: int64

#### Вывод

Имеем три таблицы с информацией о посещениях сайта  `visits`(6 столбцов, 309 901 строк), информацией о заказах `orders`(3 столбца, 40 212 строк) и информацией о затратах на маркетинг `costs`(3 столбца, 1 800 строк).

Анализ проводится по 4 странам: США, Великобритания, Франция и Германия. Категории устройств пользователя: iPhone, Android, PC, Mac.

Названия столбцов были заменены на строчные буквы и нижнее подчеркивание. Значения столбцов с датами были преобразованы из строк в формат datetime64.

Пропущенные и дублирующиеся значения в таблицах отсутствуют.

### Задайте функции для расчёта и анализа LTV, ROI, удержания и конверсии.

Разрешается использовать функции, с которыми вы познакомились в теоретических уроках.

Это функции для вычисления значений метрик:

- `get_profiles()` — для создания профилей пользователей,
- `get_retention()` — для подсчёта Retention Rate,
- `get_conversion()` — для подсчёта конверсии,
- `get_ltv()` — для подсчёта LTV.

А также функции для построения графиков:

- `filter_data()` — для сглаживания данных,
- `plot_retention()` — для построения графика Retention Rate,
- `plot_conversion()` — для построения графика конверсии,
- `plot_ltv_roi` — для визуализации LTV и ROI.

### Функция для расчёта LTV и ROI

In [ ]:
def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

### Функция для расчёта удержания

In [ ]:
def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

### Функция для расчёта конверсии

In [ ]:
def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis=1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

### Функция создания профилей

In [ ]:
def get_profiles(visits, orders, costs):

    # находим параметры первых посещений
    profiles = (
        visits.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # считаем количество уникальных пользователей с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    costs = costs.merge(new_users, on=['dt', 'channel'], how='left')
    #costs['acquisition_cost'] = costs.set_index('key').join(new_users.set_index('key'))

    # делим рекламные расходы на число привлечённых пользователей
    costs['acquisition_cost'] = costs['costs'] / costs['unique_users']
 

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles

### Функция для визуализации LTV и ROI

In [ ]:
def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(15, 15))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(3, 2, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(3, 2, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(3, 2, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(3,2, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(3, 2, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

### Функция для визуализации конверсии

In [ ]:
def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(30, 10))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()#bbox_to_anchor=(1.0, 1.05), loc='upper left')
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

### Функция для визуализации удержания

In [ ]:
def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show()

### Функция для сглаживания фрейма

In [ ]:
def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df

### Исследовательский анализ данных

- Составьте профили пользователей. Определите минимальную и максимальную даты привлечения пользователей.
- Выясните, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих из каждой страны.
- Узнайте, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого устройства.
- Изучите рекламные источники привлечения и определите каналы, из которых пришло больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

После каждого пункта сформулируйте выводы.

In [ ]:
# момент анализа
observation_date = datetime(2019, 11, 1).date()
observation_date

datetime.date(2019, 11, 1)

In [ ]:
# горизонт анализа
horizon_days = 14

In [ ]:
# создаем опцию «игнорировать горизонт»
ignore_horizon = False

In [ ]:
# минимальная и максимально возможные дата привлечения пользователей
print('Минимальная дата привлечения пользователей:', costs['dt'].min())
print('Максимальная дата привлечения пользователей:', costs['dt'].max())

Минимальная дата привлечения пользователей: 2019-05-01
Максимальная дата привлечения пользователей: 2019-10-27


Даты соответствуют ТЗ, продолжим анализ</div>

In [ ]:
# построим профили пользователей
profiles = get_profiles(visits, orders, costs)
profiles.head()

,user_id,first_ts,channel,device,region,dt,month,payer,acquisition_cost
0,599326,2019-05-07 20:58:57,FaceBoom,Mac,United States,2019-05-07,2019-05-01,True,1.088172
1,4919697,2019-07-09 12:46:07,FaceBoom,iPhone,United States,2019-07-09,2019-07-01,False,1.107237
2,6085896,2019-10-01 09:58:33,organic,iPhone,France,2019-10-01,2019-10-01,False,0.000000
3,22593348,2019-08-22 21:35:48,AdNonSense,PC,Germany,2019-08-22,2019-08-01,False,0.988235
4,31989216,2019-10-02 00:07:44,YRabbit,iPhone,United States,2019-10-02,2019-10-01,False,0.230769


In [ ]:
# смотрим страны и платящих пользователей
report_region = profiles.groupby('region').agg({'user_id': 'count', 'payer':['sum', 'mean']})
report_region.columns = ['Пользователи', 'Платящие пользователи', '% платящих пользователей']
report_region['% платящих пользователей'] = 100*report_region['% платящих пользователей']
report_region.sort_values(by='% платящих пользователей', ascending=False).style.format({'Платящие пользователи': '{:.0f}',
                                                                    '% платящих пользователей': '{:.3f}%'})

,Пользователи,Платящие пользователи,% платящих пользователей
region,,,
United States,100002,6902,6.902%
Germany,14981,616,4.112%
UK,17575,700,3.983%
France,17450,663,3.799%


In [ ]:
# смотрим разбивку по устройствам и платящих пользователей
report_device = profiles.groupby('device').agg({'user_id': 'count', 'payer': ['sum', 'mean']})
report_device.columns = ['Пользователи', 'Платящие пользователи', '% платящих пользователей']
report_device['% платящих пользователей'] = 100*report_device['% платящих пользователей']
report_device.sort_values(by='% платящих пользователей', ascending=False).style.format({'Платящие пользователи': '{:.0f}',
                                                                    '% платящих пользователей': '{:.3f}%'})


,Пользователи,Платящие пользователи,% платящих пользователей
device,,,
Mac,30042,1912,6.364%
iPhone,54479,3382,6.208%
Android,35032,2050,5.852%
PC,30455,1537,5.047%


In [ ]:
# смотрим разбивку по каналам и платящих пользователей
report_channel = profiles.groupby('channel').agg({'user_id': 'count', 'payer': ['sum', 'mean']})
report_channel.columns = ['Пользователи', 'Платящие пользователи', '% платящих пользователей']
report_channel['% платящих пользователей'] = 100*report_channel['% платящих пользователей']
report_channel.sort_values(by='% платящих пользователей', ascending=False).style.format({'Платящие пользователи': '{:.0f}',
                                                                    '% платящих пользователей': '{:.3f}%'})

,Пользователи,Платящие пользователи,% платящих пользователей
channel,,,
FaceBoom,29144,3557,12.205%
AdNonSense,3880,440,11.340%
lambdaMediaAds,2149,225,10.470%
TipTop,19561,1878,9.601%
RocketSuperAds,4448,352,7.914%
WahooNetBanner,8553,453,5.296%
YRabbit,4312,165,3.827%
MediaTornado,4364,156,3.575%
LeapBob,8553,262,3.063%


#### Вывод

Мы посмотрели на базовые значения конверсии пользователей в покупатели по регионам, устройствам и каналу привлечения:

Больше всего приходит пользователей из США и они лучше других конвертируется (почти 7%);
Наибольшая конверсия у Macоводов, но в целом пользователи iphone и android отстали не сильно (у всех почти 6%), у PC всего 5%;
Самые активные каналы привлечения FaceBoom, AdNonSense и lambdaMediaAds, у них платящих пользователей более 10%, органические клиенты - самые неплатящие, всего около 2%)

### Маркетинг

- Посчитайте общую сумму расходов на маркетинг.
- Выясните, как траты распределены по рекламным источникам, то есть сколько денег потратили на каждый источник.
- Постройте визуализацию динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику. Постарайтесь отразить это на одном графике.
- Узнайте, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника. Используйте профили пользователей.

Напишите промежуточные выводы.

#### Общая сумму расходов на маркетинг

In [ ]:
# всего потрачено денег
round(costs.costs.sum())

105497

In [ ]:
# потрачено денег на каждый источник
report_costs = profiles.groupby('channel').agg({'user_id': 'nunique', 'acquisition_cost': ['sum', 'mean']})
report_costs.columns = ['Пользователи', 'Всего израсходовано', 'В среднем на пользователя']
report_costs['% от общих затрат'] = report_costs['Всего израсходовано'] / report_costs['Всего израсходовано'].sum() * 100

report_costs = report_costs.sort_values(by='% от общих затрат', ascending=False).reset_index().style.format(
    {'В среднем на пользователя': '{:.2f}',
    '% от общих затрат': '{:.2f}'})

report_costs

,channel,Пользователи,Всего израсходовано,В среднем на пользователя,% от общих затрат
0,TipTop,19561,54751.300000,2.80,51.90
1,FaceBoom,29144,32445.600000,1.11,30.75
2,WahooNetBanner,8553,5151.000000,0.60,4.88
3,AdNonSense,3880,3911.250000,1.01,3.71
4,OppleCreativeMedia,8605,2151.250000,0.25,2.04
5,RocketSuperAds,4448,1833.000000,0.41,1.74
6,LeapBob,8553,1797.600000,0.21,1.70
7,lambdaMediaAds,2149,1557.600000,0.72,1.48
8,MediaTornado,4364,954.480000,0.22,0.90
9,YRabbit,4312,944.220000,0.22,0.90


In [ ]:
print('Средняя стоимость привлечения одного пользователя:', 
      round(profiles.query('channel != "organic"')['acquisition_cost'].mean(), 2))

Средняя стоимость привлечения одного пользователя: 1.13


In [ ]:
# визуализация изменений трат во времени
costs.pivot_table(
    index='dt',  # даты 
    columns='channel',  # источники переходов
    values='costs',  
    aggfunc='sum',  
).plot(figsize=(15, 10), grid=True)

plt.title('Изменение трат по времени')
plt.xlabel('Месяц')
plt.ylabel('Расходы')
plt.show()

In [ ]:
profiles.pivot_table(index='channel', 
    columns='month', 
    values='acquisition_cost', 
    aggfunc='sum'
).T.plot(figsize=(10, 5), grid=True)

plt.legend()
plt.xlabel('Месяц')
plt.title('Затраты на рекламу')
plt.show()

In [ ]:
# Стоимость привлечения одного покупателя из каждого источника в среднем
profiles.pivot_table(
    index='dt', columns='channel', values='acquisition_cost', aggfunc='mean'
).plot(grid=True, figsize=(15, 10))
plt.ylabel('CAC')
plt.xlabel('Дата привлечения')
plt.title('Динамика САС по каналам привлечения')
plt.legend(bbox_to_anchor=(1.02, 1.0), loc='best')
plt.show()

#### Вывод

 По затратам на рекламу лидирует канал TipTop  (затраты на маркетинг более половины от общей суммы). Почти треть затрат приходится на канал FaceBoom, но стоимость привлечения пользователя заметно ниже, чем у лидера. По остальным каналам траты не такие большие, и в отличие от лидеров незначительно изменялись со временем.  
 Стоимость привлечения пользователя также самая высокая у TipTop и составляет 2,8 у.е., все остальные затрачивают существенно меньшее количество денег на одного пользователя, в среднем от 0,22 до 1,11 у.е.

### Оцените окупаемость рекламы

Используя графики LTV, ROI и CAC, проанализируйте окупаемость рекламы. Считайте, что на календаре 1 ноября 2019 года, а в бизнес-плане заложено, что пользователи должны окупаться не позднее чем через две недели после привлечения. Необходимость включения в анализ органических пользователей определите самостоятельно.

- Проанализируйте окупаемость рекламы c помощью графиков LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проверьте конверсию пользователей и динамику её изменения. То же самое сделайте с удержанием пользователей. Постройте и изучите графики конверсии и удержания.
- Проанализируйте окупаемость рекламы с разбивкой по устройствам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по странам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по рекламным каналам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Ответьте на такие вопросы:
    - Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Чем могут быть вызваны проблемы окупаемости?

Напишите вывод, опишите возможные причины обнаруженных проблем и промежуточные рекомендации для рекламного отдела.

In [ ]:
# исключим органических пользователей из таблицы
profiles_ads = profiles.query('channel != "organic"')

In [ ]:
# посчитаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles_ads, orders, observation_date, horizon_days
)

# построим графики
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days)

По графикам можно сделать такие выводы:
- Реклама не окупается, ROI находится ниже уровня окупаемости
- С июля пользователь перестали окупаться
- LTV постепенно растет, что радует
- САС также постоянно растет, не радует

### Анализ окупаемости рекламы с разбивкой по устройствам

In [ ]:
# смотрим окупаемость с разбивкой по устройствам

dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles_ads, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
)

LTV стабильно растет на всех устройствах  
Окупаются только пользователи PC. На графике заметно снижение динамики ROI у всех устройств при росте стоимости привлечения пользователей.  
К сентябрю пользователи всех устройств перестали и приносить прибыль

### Анализ окупаемости рекламы с разбивкой по странам

In [ ]:
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles_ads, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
)

Выручка пользователей США стабильно выше пользователей других стран но, в то же время только у США ROI не достигает уровня окупаемости.  
Динамика стоимости привлечения пользователей показывает огромный разрыв расходов в США по сравнению с другими странами, стоимость в США в разы выше.  
Динамика LTV подвержена сезонности и стабильна. LTV в США выше, чем в других странах.

### Окупаемость рекламы с разбивкой по рекламным каналам

In [ ]:
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles_ads, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

Не окупилась реклама 3 каналов: TipTop, FaceBoom и AdNonSense  
Затраты на рекламу в TipTop постоянно растут, но не окупаются  
По стоимости привлечения сильно выбивается канал Tip Top, как видим большие расходы не были оправданы

In [ ]:
# конверсия с разбивкой по устройствам
conversion_raw, conversion, conversion_history = get_conversion(
    profiles_ads,
    orders,
    observation_date,
    horizon_days,
    dimensions=['device'],
    ignore_horizon=False,
)
plot_conversion(conversion, conversion_history, horizon_days, window=7)

In [ ]:
# удержание с разбивкой по устройствам
retention_raw, retention, retention_history = get_retention(
    profiles_ads,
    visits,
    observation_date,
    horizon_days,
    dimensions=['device'],
    ignore_horizon=False,
)
plot_retention(retention, retention_history, horizon_days, window=7)

Лучшая конверсия у maca и iphone. Конверсия пользователей PC заметно ниже остальных, в то время, как удержание - наоборот самое высокое.

In [ ]:
# конверсия с разбивкой по регионам
conversion_raw, conversion, conversion_history = get_conversion(
    profiles_ads,
    orders,
    observation_date,
    horizon_days,
    dimensions=['region'],
    ignore_horizon=False,
)
plot_conversion(conversion, conversion_history, horizon_days, window=7)

In [ ]:
# удержание с разбивкой по регионам
retention_raw, retention, retention_history = get_retention(
    profiles_ads,
    visits,
    observation_date,
    horizon_days,
    dimensions=['region'],
    ignore_horizon=False,
)
plot_retention(retention, retention_history, horizon_days, window=7)

Конверсия у пользователи США почти в 2 раза выше остальных, но удержание худшее из всех. Европейцы стабильно конвертируются и удерживаются

In [ ]:
# конверсия с разбивкой по каналам
conversion_raw, conversion, conversion_history = get_conversion(
    profiles_ads,
    orders,
    observation_date,
    horizon_days,
    dimensions=['channel'],
    ignore_horizon=False,
)
plot_conversion(conversion, conversion_history, horizon_days, window=7)

In [ ]:
# удержание с разбивкой по каналам
retention_raw, retention, retention_history = get_retention(
    profiles_ads,
    visits,
    observation_date,
    horizon_days,
    dimensions=['channel'],
    ignore_horizon=False,
)
plot_retention(retention, retention_history, horizon_days, window=7)

Cамая высокая конверсия у пользователей канала FaceBoom и AdNonSense, в то же время график показывает худшее удержание.  
Пользователи TipTop конвертируются хуже, но хорошо удерживаются. В остальных каналах нет проблем с конверсией и удержанием

Удержание неплатящих пользователей из разных каналов, стран, платформ ровное, а удержание платящих сильно отличается. Вероятно, количество платящих пользователей существенно ниже и этим объясняется различие в "поведении" графиков.

### Выводы

Чаще всего приложением пользуются пользователи из США, пришедшие через TipTop и FaceBoom, на привлечение которых было потрачено 80% от общей суммы всех затрат на рекламу, а также жители Германии, Франции и Великобритании. Приложение доступно на таких устройствах как Mac, iPhone, Android, PC.

В результате исследования было выявлено, что реклама не окупается в США. Неэффективными оказались каналы TipTop, FaceBoom и AdNonSense. Выручка от пользователей, пришедших через TipTop не компенсировала затраты на рекламу, которые постоянно росли в течение всего периода. LTV пользователей, пришедших через FaceBoom и AdNonSense стабильно, но они стабильно плохо удерживались. Вид устройства никак не повлиял на окупаемость.  

Рекламному отделу развлекательного приложения Procrastinate Pro+ рекомендуется сократить финансирование на маркетинг там, где стоимость привлечения слишком большая - это расходы на пользователей из США и канал TipTop и FaceBoom.
Рассмотреть возможность привлечения большего количества пользователей стран Европы, где реклама стабильно окупается

По `Yrabbit` видим лучшую окупаемость, средненьку конверсию и слабоватое удержание платящих пользователей, ltv растет слабо, а затраты на рекламу минимальны, можно поработать, возможные доп вливания в рекламу вероятно поднимут нужные нам метрики.  
    
У `RocketSuperAds` аналогично ROI позитвное, по конверсии крепкий середнячок, удержание одно из лучших, стоимость привлечения не самая высокая, можно чуть увеличить рекламный бюджет</div>